In [14]:
HOST = "cluster0.oostpcw.mongodb.net"
USER = "tonyjqi"
PW="9qAJpagY83EymCQT"
DB="final_project"

In [15]:
from pymongo import MongoClient

def connect():

    url = "mongodb+srv://{}:{}@{}".format(USER, PW, HOST)
    connection = MongoClient(url)
    return connection[DB]

In [16]:
connection = connect()

print(connection)

Database(MongoClient(host=['ac-esnplxm-shard-00-02.oostpcw.mongodb.net:27017', 'ac-esnplxm-shard-00-00.oostpcw.mongodb.net:27017', 'ac-esnplxm-shard-00-01.oostpcw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-2oveo8-shard-0', tls=True), 'final_project')


In [56]:
from faker import Faker
from faker_airtravel import AirTravelProvider
from random import randint, uniform
from datetime import datetime, time

database = connection[DB] 
collection = connection["ticketing"]

database.collection.create_index("tck_id", unique=True)

fake = Faker()
fake.add_provider(AirTravelProvider)

Faker.seed(1234)

possible_cust_ids = list(range(1, 51))

def generate_date():
    return fake.date_this_decade()

def generate_time():
    return fake.time()

common_date = datetime.strptime("0001-01-01", "%Y-%m-%d").date()

records = []

for _ in range(100):
    dep_date = generate_date()
    dep_time = datetime.strptime(generate_time(), "%H:%M:%S").time()

    arr_date = generate_date()
    arr_time = datetime.strptime(generate_time(), "%H:%M:%S").time()
    
    cust_id = fake.random_element(possible_cust_ids)
    
    flight_data = fake.flight()
    
    rec = {
        "tck_id": str(fake.uuid4()),
        "cust_id": cust_id,
        "airline": flight_data['airline'],
        "flight_nm": fake.random_int(min=100, max=999),
        "dep_airport": flight_data['origin']['airport'],
        "arr_airport": flight_data['destination']['airport'],
        "dep_date": datetime.combine(dep_date, dep_time),
        "dep_time": datetime(common_date.year, common_date.month, common_date.day, dep_time.hour, dep_time.minute, dep_time.second),
        "arr_date": datetime.combine(arr_date, arr_time),
        "arr_time": datetime(common_date.year, common_date.month, common_date.day, arr_time.hour, arr_time.minute, arr_time.second),
        "stops": flight_data['stops'],
        "tik_amt": flight_data['price'],
        "curr_code": fake.currency_code(),
    }

    records.append(rec)
    
try:
    connection = connect()
    collection = connection["ticketing"]
    collection.insert_many(records)
    print("100 documents written into ticketing collection")
    
except Exception as e:
    print("Error occurred while writing documents:", e)

100 documents written into ticketing collection
